# Imports

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import pipeline
from sklearn.pipeline import Pipeline

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# Settings

In [10]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Load Data

In [11]:
PATH_DATA = "data/data.csv"
PATH_FEATURE = "data/features.csv"
TARGET = 'quantity_of_paid'

df_data = pd.read_csv(PATH_DATA).iloc[:10000, :]
df_feature = pd.read_csv(PATH_FEATURE).iloc[:10000, :]

In [12]:
print(np.unique(df_data['sku']))
df_feature.head(5)

[202562694 202656346]


,date,sales,real_price,gds_net_profit_after_tax,fbo_stocks,fbs_stocks,sales_profit,gross_profit,premium_service_cost,cross_docking_cost_wght_avg_rate,placement_cost_wght_avg_rate,gds_tax,direct_logistics,installment_sale_cost,last_mile,acquiring_cost_wght_avg_rate,orders_commission,returns_commission,other_wght_avg_rate,cancellation_processing,promotion_advertising_wght_avg_rate,promotion_reviews_wght_avg_rate,reverse_logistics,seller_bonuses_cost,shipment_processing,revenue_for_date,day,month,weekday,is_weekend,is_holiday,season_spring,season_summer,norm_price,sales_shift_1,sales_shift_2,sales_shift_3,cycles_sales,trend_sales,SES12_sales,TESadd12_sales,ma_sales_3,sma_sales_3,macd_sales
0,2023-05-01,2528,407.295366,117.45907,212.678841,0.0,168.708761,299.106395,0.0,-7.113075,0.0,-28.466352,-47.625248,-0.366701,-23.305899,-4.088636,-56.293986,0.227503,-1.333032,-1.190949,-10.248596,0.0,-1.105200,-0.737154,0.0,1.029643e+06,1,5,0,0,0,1,0,1.000049,2527.0,2528.0,2528.0,0.373654,2527.626346,2528.000000,2527.703956,0.000000,2528.000000,0.000000
1,2023-05-02,2527,407.376886,117.51552,212.635028,0.0,168.775357,299.186660,0.0,-7.114595,0.0,-28.471903,-47.625496,-0.366847,-23.307212,-4.089409,-56.304833,0.227593,-1.333257,-1.191422,-10.250675,0.0,-1.105639,-0.737446,0.0,1.029441e+06,2,5,1,0,0,1,0,1.000250,2528.0,2528.0,2528.0,-0.735261,2527.735261,2527.846154,2527.974730,0.000000,2527.333333,-0.079772
2,2023-05-03,2528,407.295366,117.45907,212.678841,0.0,168.708761,299.106395,0.0,-7.113075,0.0,-28.466352,-47.625248,-0.366701,-23.305899,-4.088636,-56.293986,0.227503,-1.333032,-1.190949,-10.248596,0.0,-1.105200,-0.737154,0.0,1.029643e+06,3,5,2,0,0,1,0,1.000049,2528.0,2528.0,2528.0,0.155591,2527.844409,2527.869822,2527.355523,2527.666667,2527.714286,-0.061590
3,2023-05-04,2528,407.295366,117.45907,212.678841,0.0,168.708761,299.106395,0.0,-7.113075,0.0,-28.466352,-47.625248,-0.366701,-23.305899,-4.088636,-56.293986,0.227503,-1.333032,-1.190949,-10.248596,0.0,-1.105200,-0.737154,0.0,1.029643e+06,4,5,3,0,0,1,0,1.000049,2528.0,2528.0,2527.0,0.046436,2527.953564,2527.889850,2527.806514,2527.666667,2527.866667,-0.046644
4,2023-05-05,2528,407.295366,117.45907,212.678841,0.0,168.708761,299.106395,0.0,-7.113075,0.0,-28.466352,-47.625248,-0.366701,-23.305899,-4.088636,-56.293986,0.227503,-1.333032,-1.190949,-10.248596,0.0,-1.105200,-0.737154,0.0,1.029643e+06,5,5,4,0,0,1,0,1.000049,2528.0,2527.0,2528.0,-0.062598,2528.062598,2527.906796,2527.791537,2528.000000,2527.935484,-0.034402


In [13]:
df_data['sku'] != df_data['sku'][0]

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
30      False
31      False
32      False
33      False
34      False
35      False
36      False
37      False
38      False
39      False
40      False
41      False
42      False
43      False
44      False
45      False
46      False
47      False
48      False
49      False
50      False
51      False
52      False
53      False
54      False
55      False
56      False
57      False
58      False
59      False
60      False
61      False
62      False
63      False
64      False
65      False
66      False
67      False
68      False
69      False
70      False
71    

In [14]:
df_data[(df_data['sku'] == df_data['sku'][0]) & (df_data['date_x'] == df_data['date_x'][0])].head()


,id_x,shop_id_x,date_x,sku,hits_view_search,hits_view_pdp,hits_view,hits_tocart,postings,postings_premium,ordered_units,delivered_units,revenue,cancellations,returns,adv_sum_all,position_category,id_y,shop_id_y,date_y,delivery_schema,warehouse_name,offer_id,title,quantity_of_paid,quantity_of_return,accruals_for_orders,accruals_for_return,prime_cost_of_paid,prime_cost_of_return,gross_profit,orders_commission,returns_commission,shipment_processing,direct_logistics,last_mile,cancellation_processing,reverse_logistics,premium_service_cost,installment_sale_cost,seller_bonuses_cost,sales_profit,compensations,compensations_wght_avg_rate,promotion_advertising,promotion_advertising_wght_avg_rate,promotion_reviews,promotion_reviews_wght_avg_rate,cross_docking_cost,cross_docking_cost_wght_avg_rate,placement_cost,placement_cost_wght_avg_rate,acquiring_cost,acquiring_cost_wght_avg_rate,other,other_wght_avg_rate,shop_net_profit_bfr_tax,gds_net_profit_bfr_tax,shop_tax,gds_tax,shop_net_profit_after_tax,gds_net_profit_after_tax,shop_profitability_after_tax,gds_profitability_after_tax,cost_price,fbo_stocks,fbs_stocks
0,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202562694,442,63,2491,8,3.0,0.0,3,0,1482.0,0,0,55.71,64.717195,2726400,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-29,FBO,ПУШКИНО,D200,"Бутылка стеклянная, емкость для масла с пробко...",1,0,473.0,0.0,-107.0,0.0,366.0,-66.22,0.0,0.0,-45.0,-26.02,0.0,0.0,0.0,0.0,-1.25,227.51,0.0,0.0,0.0,-11.820688,0.0,0.0,0.0,-9.679734,0.0,0.0,0.0,-4.632474,0.0,-1.567194,227.51,199.809911,-33.11,-33.11,194.40,166.699911,41.10,35.24,107.0,293,0
1,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202562694,442,63,2491,8,3.0,0.0,3,0,1482.0,0,0,55.71,64.717195,2726690,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-01,FBO,ПУШКИНО,D200,"Бутылка стеклянная, емкость для масла с пробко...",1,0,473.0,0.0,-107.0,0.0,366.0,-66.22,0.0,0.0,-45.0,-26.02,0.0,0.0,0.0,0.0,0.00,228.76,0.0,0.0,0.0,-16.049131,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,-4.973144,0.0,0.000000,228.76,207.737724,-33.11,-33.11,195.65,174.627724,41.36,36.92,107.0,293,0
2,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202562694,442,63,2491,8,3.0,0.0,3,0,1482.0,0,0,55.71,64.717195,2724273,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-01,FBO,НИЖНИЙ НОВГОРОД,D200,"Бутылка стеклянная, емкость для масла с пробко...",5,0,2340.0,0.0,-535.0,0.0,1805.0,-280.80,0.0,0.0,-225.0,-128.70,0.0,0.0,0.0,0.0,-1.25,1169.25,0.0,0.0,0.0,-56.665721,0.0,0.0,0.0,-37.162130,0.0,0.0,0.0,-24.224829,0.0,-8.680882,1169.25,1042.516438,-163.80,-163.80,1005.45,878.716438,42.97,37.55,107.0,293,0
3,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202562694,442,63,2491,8,3.0,0.0,3,0,1482.0,0,0,55.71,64.717195,2724274,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-01,FBO,НИЖНИЙ НОВГОРОД,D200,"Бутылка стеклянная, емкость для масла с пробко...",1,0,468.0,0.0,-107.0,0.0,361.0,-56.16,0.0,0.0,-45.0,-25.74,0.0,0.0,0.0,0.0,-1.25,232.85,0.0,0.0,0.0,-11.333144,0.0,0.0,0.0,-7.432426,0.0,0.0,0.0,-4.844966,0.0,-1.736176,232.85,207.503288,-32.76,-32.76,200.09,174.743288,42.75,37.34,107.0,293,0
4,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202562694,442,63,2491,8,3.0,0.0,3,0,1482.0,0,0,55.71,64.717195,2724275,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-01,FBO,НИЖНИЙ НОВГОРОД,D200,"Бутылка стеклянная, емкость для масла с пробко...",1,0,468.0,0.0,-107.0,0.0,361.0,-56.16,0.0,0.0,-45.0,-25.74,0.0,0.0,0.0,0.0,0.00,234.10,0.0,0.0,0.0,-11.333144,0.0,0.0,0.0,-7.432426,0.0,0.0,0.0,-4.844966,0.0,-1.736176,234.10,208.753288,-32.76,-32.76,201.34,175.993288,43.02,37.61,107.0,293,0


In [15]:
df_data[(df_data['sku'] != df_data['sku'][0]) & (df_data['date_x'] == df_data['date_x'][0])].head()

,id_x,shop_id_x,date_x,sku,hits_view_search,hits_view_pdp,hits_view,hits_tocart,postings,postings_premium,ordered_units,delivered_units,revenue,cancellations,returns,adv_sum_all,position_category,id_y,shop_id_y,date_y,delivery_schema,warehouse_name,offer_id,title,quantity_of_paid,quantity_of_return,accruals_for_orders,accruals_for_return,prime_cost_of_paid,prime_cost_of_return,gross_profit,orders_commission,returns_commission,shipment_processing,direct_logistics,last_mile,cancellation_processing,reverse_logistics,premium_service_cost,installment_sale_cost,seller_bonuses_cost,sales_profit,compensations,compensations_wght_avg_rate,promotion_advertising,promotion_advertising_wght_avg_rate,promotion_reviews,promotion_reviews_wght_avg_rate,cross_docking_cost,cross_docking_cost_wght_avg_rate,placement_cost,placement_cost_wght_avg_rate,acquiring_cost,acquiring_cost_wght_avg_rate,other,other_wght_avg_rate,shop_net_profit_bfr_tax,gds_net_profit_bfr_tax,shop_tax,gds_tax,shop_net_profit_after_tax,gds_net_profit_after_tax,shop_profitability_after_tax,gds_profitability_after_tax,cost_price,fbo_stocks,fbs_stocks
3604,10373,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202656346,539,113,6005,10,4.0,1.0,4,3,1908.0,0,0,131.73,38.762523,2726385,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-29,FBO,ПУШКИНО,G500-2,"Бутылка стеклянная ""БТ"", емкость для масла с п...",1,0,477.0,0.0,-108.0,0.0,369.0,-66.78,0.0,0.0,-47.0,-26.24,0.0,0.0,0.0,0.00,-1.25,227.73,0.0,0.0,0.0,-11.920651,0.0,0.0,0.0,-9.761592,0.0,0.0,0.0,-4.671649,0.0,-1.580447,227.73,199.795661,-33.39,-33.39,194.34,166.405661,40.74,34.89,108.0,384,0
3605,10373,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202656346,539,113,6005,10,4.0,1.0,4,3,1908.0,0,0,131.73,38.762523,2726388,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-29,FBO,ПУШКИНО,G500-2,"Бутылка стеклянная ""БТ"", емкость для масла с п...",0,0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,-47.0,0.00,-50.0,-47.0,0.0,0.00,0.00,-144.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,-144.00,-144.000000,0.00,0.00,-144.00,-144.000000,0.00,0.00,108.0,384,0
3606,10373,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202656346,539,113,6005,10,4.0,1.0,4,3,1908.0,0,0,131.73,38.762523,2726403,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-29,FBO,ПУШКИНО,G500-2,"Бутылка стеклянная ""БТ"", емкость для масла с п...",1,0,412.0,0.0,-108.0,0.0,304.0,-57.68,0.0,0.0,-47.0,-22.66,0.0,0.0,0.0,-44.88,-1.25,130.53,0.0,0.0,0.0,-10.296244,0.0,0.0,0.0,-8.431396,0.0,0.0,0.0,-4.035051,0.0,-1.365082,130.53,106.402227,-28.84,-28.84,101.69,77.562227,24.68,18.83,108.0,384,0
3607,10373,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202656346,539,113,6005,10,4.0,1.0,4,3,1908.0,0,0,131.73,38.762523,2726405,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-29,FBO,РОСТОВ-НА-ДОНУ,G500-2,"Бутылка стеклянная ""БТ"", емкость для масла с п...",1,0,444.0,0.0,-108.0,0.0,336.0,-62.16,0.0,0.0,-47.0,-24.42,0.0,0.0,0.0,0.00,-1.25,201.17,0.0,0.0,0.0,-11.095952,0.0,0.0,0.0,-9.086262,0.0,0.0,0.0,-4.348453,0.0,-1.471108,201.17,175.168225,-31.08,-31.08,170.09,144.088225,38.31,32.45,108.0,384,0
3608,10373,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202656346,539,113,6005,10,4.0,1.0,4,3,1908.0,0,0,131.73,38.762523,2726445,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-29,FBO,САМАРА,G500-2,"Бутылка стеклянная ""БТ"", емкость для масла с п...",1,0,444.0,0.0,-108.0,0.0,336.0,-62.16,0.0,0.0,-47.0,-24.42,0.0,0.0,0.0,0.00,-1.25,201.17,0.0,0.0,0.0,-11.095952,0.0,0.0,0.0,-9.086262,0.0,0.0,0.0,-4.348453,0.0,-1.471108,201.17,175.168225,-31.08,-31.08,170.09,144.088225,38.31,32.45,108.0,384,0


In [16]:
df_data[(df_data['sku'] == '202656346') & (df_data['date_x'] == '2023-06-03')]


,id_x,shop_id_x,date_x,sku,hits_view_search,hits_view_pdp,hits_view,hits_tocart,postings,postings_premium,ordered_units,delivered_units,revenue,cancellations,returns,adv_sum_all,position_category,id_y,shop_id_y,date_y,delivery_schema,warehouse_name,offer_id,title,quantity_of_paid,quantity_of_return,accruals_for_orders,accruals_for_return,prime_cost_of_paid,prime_cost_of_return,gross_profit,orders_commission,returns_commission,shipment_processing,direct_logistics,last_mile,cancellation_processing,reverse_logistics,premium_service_cost,installment_sale_cost,seller_bonuses_cost,sales_profit,compensations,compensations_wght_avg_rate,promotion_advertising,promotion_advertising_wght_avg_rate,promotion_reviews,promotion_reviews_wght_avg_rate,cross_docking_cost,cross_docking_cost_wght_avg_rate,placement_cost,placement_cost_wght_avg_rate,acquiring_cost,acquiring_cost_wght_avg_rate,other,other_wght_avg_rate,shop_net_profit_bfr_tax,gds_net_profit_bfr_tax,shop_tax,gds_tax,shop_net_profit_after_tax,gds_net_profit_after_tax,shop_profitability_after_tax,gds_profitability_after_tax,cost_price,fbo_stocks,fbs_stocks


In [17]:
(df_data['sku'] == '202562694')  (df_data['date_x'] == '2023-06-03')

TypeError: 'Series' object is not callable

In [18]:
(df_data['date_x'] == '2023-06-03')

0        True
1        True
2        True
3        True
4        True
5        True
6        True
7        True
8        True
9        True
10       True
11       True
12       True
13       True
14       True
15       True
16       True
17       True
18       True
19       True
20       True
21       True
22       True
23       True
24       True
25       True
26       True
27       True
28       True
29       True
30       True
31       True
32       True
33       True
34       True
35       True
36       True
37       True
38       True
39       True
40       True
41       True
42       True
43       True
44       True
45       True
46       True
47       True
48       True
49       True
50       True
51       True
52       True
53       True
54       True
55       True
56       True
57       True
58       True
59       True
60       True
61       True
62       True
63       True
64       True
65       True
66       True
67       True
68       True
69       True
70       True
71    

In [19]:
# set(df_data.columns) - set(df_feature.columns)

In [20]:
df_feature.head(5)

,date,sales,real_price,gds_net_profit_after_tax,fbo_stocks,fbs_stocks,sales_profit,gross_profit,premium_service_cost,cross_docking_cost_wght_avg_rate,placement_cost_wght_avg_rate,gds_tax,direct_logistics,installment_sale_cost,last_mile,acquiring_cost_wght_avg_rate,orders_commission,returns_commission,other_wght_avg_rate,cancellation_processing,promotion_advertising_wght_avg_rate,promotion_reviews_wght_avg_rate,reverse_logistics,seller_bonuses_cost,shipment_processing,revenue_for_date,day,month,weekday,is_weekend,is_holiday,season_spring,season_summer,norm_price,sales_shift_1,sales_shift_2,sales_shift_3,cycles_sales,trend_sales,SES12_sales,TESadd12_sales,ma_sales_3,sma_sales_3,macd_sales
0,2023-05-01,2528,407.295366,117.45907,212.678841,0.0,168.708761,299.106395,0.0,-7.113075,0.0,-28.466352,-47.625248,-0.366701,-23.305899,-4.088636,-56.293986,0.227503,-1.333032,-1.190949,-10.248596,0.0,-1.105200,-0.737154,0.0,1.029643e+06,1,5,0,0,0,1,0,1.000049,2527.0,2528.0,2528.0,0.373654,2527.626346,2528.000000,2527.703956,0.000000,2528.000000,0.000000
1,2023-05-02,2527,407.376886,117.51552,212.635028,0.0,168.775357,299.186660,0.0,-7.114595,0.0,-28.471903,-47.625496,-0.366847,-23.307212,-4.089409,-56.304833,0.227593,-1.333257,-1.191422,-10.250675,0.0,-1.105639,-0.737446,0.0,1.029441e+06,2,5,1,0,0,1,0,1.000250,2528.0,2528.0,2528.0,-0.735261,2527.735261,2527.846154,2527.974730,0.000000,2527.333333,-0.079772
2,2023-05-03,2528,407.295366,117.45907,212.678841,0.0,168.708761,299.106395,0.0,-7.113075,0.0,-28.466352,-47.625248,-0.366701,-23.305899,-4.088636,-56.293986,0.227503,-1.333032,-1.190949,-10.248596,0.0,-1.105200,-0.737154,0.0,1.029643e+06,3,5,2,0,0,1,0,1.000049,2528.0,2528.0,2528.0,0.155591,2527.844409,2527.869822,2527.355523,2527.666667,2527.714286,-0.061590
3,2023-05-04,2528,407.295366,117.45907,212.678841,0.0,168.708761,299.106395,0.0,-7.113075,0.0,-28.466352,-47.625248,-0.366701,-23.305899,-4.088636,-56.293986,0.227503,-1.333032,-1.190949,-10.248596,0.0,-1.105200,-0.737154,0.0,1.029643e+06,4,5,3,0,0,1,0,1.000049,2528.0,2528.0,2527.0,0.046436,2527.953564,2527.889850,2527.806514,2527.666667,2527.866667,-0.046644
4,2023-05-05,2528,407.295366,117.45907,212.678841,0.0,168.708761,299.106395,0.0,-7.113075,0.0,-28.466352,-47.625248,-0.366701,-23.305899,-4.088636,-56.293986,0.227503,-1.333032,-1.190949,-10.248596,0.0,-1.105200,-0.737154,0.0,1.029643e+06,5,5,4,0,0,1,0,1.000049,2528.0,2527.0,2528.0,-0.062598,2528.062598,2527.906796,2527.791537,2528.000000,2527.935484,-0.034402


In [21]:
df_data = pd.read_csv(PATH_DATA, parse_dates=['date_x', 'date_y']).iloc[:1000, :]
df_data = pd.read_csv(PATH_DATA).iloc[:10000, :]

cols = ['date_x', 'id_x', 'shop_id_x', 'date_y', 'id_y', 'shop_id_y']
df_data[cols].head()

,date_x,id_x,shop_id_x,date_y,id_y,shop_id_y
0,2023-06-03,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-29,2726400,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe
1,2023-06-03,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-01,2726690,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe
2,2023-06-03,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-01,2724273,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe
3,2023-06-03,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-01,2724274,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe
4,2023-06-03,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-01,2724275,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe


In [22]:
df_data[df_data['date_x'] == '2023-06-03'].head(1)

,id_x,shop_id_x,date_x,sku,hits_view_search,hits_view_pdp,hits_view,hits_tocart,postings,postings_premium,ordered_units,delivered_units,revenue,cancellations,returns,adv_sum_all,position_category,id_y,shop_id_y,date_y,delivery_schema,warehouse_name,offer_id,title,quantity_of_paid,quantity_of_return,accruals_for_orders,accruals_for_return,prime_cost_of_paid,prime_cost_of_return,gross_profit,orders_commission,returns_commission,shipment_processing,direct_logistics,last_mile,cancellation_processing,reverse_logistics,premium_service_cost,installment_sale_cost,seller_bonuses_cost,sales_profit,compensations,compensations_wght_avg_rate,promotion_advertising,promotion_advertising_wght_avg_rate,promotion_reviews,promotion_reviews_wght_avg_rate,cross_docking_cost,cross_docking_cost_wght_avg_rate,placement_cost,placement_cost_wght_avg_rate,acquiring_cost,acquiring_cost_wght_avg_rate,other,other_wght_avg_rate,shop_net_profit_bfr_tax,gds_net_profit_bfr_tax,shop_tax,gds_tax,shop_net_profit_after_tax,gds_net_profit_after_tax,shop_profitability_after_tax,gds_profitability_after_tax,cost_price,fbo_stocks,fbs_stocks
0,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202562694,442,63,2491,8,3.0,0.0,3,0,1482.0,0,0,55.71,64.717195,2726400,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-29,FBO,ПУШКИНО,D200,"Бутылка стеклянная, емкость для масла с пробко...",1,0,473.0,0.0,-107.0,0.0,366.0,-66.22,0.0,0.0,-45.0,-26.02,0.0,0.0,0.0,0.0,-1.25,227.51,0.0,0.0,0.0,-11.820688,0.0,0.0,0.0,-9.679734,0.0,0.0,0.0,-4.632474,0.0,-1.567194,227.51,199.809911,-33.11,-33.11,194.4,166.699911,41.1,35.24,107.0,293,0


In [23]:
aggregation_functions = {'quantity_of_paid': 'sum', 'quantity_of_return': 'sum', 'sku': 'first'}
df_new = df_data.groupby(df_data['date_y']).aggregate(aggregation_functions)

In [24]:
df_new.head()

,quantity_of_paid,quantity_of_return,sku
date_y,,,
2023-05-01,572,-18,202562694
2023-05-02,361,0,202562694
2023-05-03,203,0,202562694
2023-05-04,441,0,202562694
2023-05-05,377,0,202562694


In [25]:
df_data.loc[:, ['date_x', 'date_y']].head()

,date_x,date_y
0,2023-06-03,2023-05-29
1,2023-06-03,2023-06-01
2,2023-06-03,2023-05-01
3,2023-06-03,2023-05-01
4,2023-06-03,2023-05-01


In [26]:
y = df_data[TARGET]
X = df_data.drop(columns=[TARGET], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle=False)
print(X_train.shape, X_test.shape)

(8000, 66) (2000, 66)


In [27]:
X_train.select_dtypes(include='object').columns

Index(['shop_id_x', 'date_x', 'shop_id_y', 'date_y', 'delivery_schema',
       'warehouse_name', 'offer_id', 'title'],
      dtype='object')

In [28]:
X_train.nunique().sort_values(ascending=False)

id_y                                   878
gds_net_profit_after_tax                76
gds_net_profit_bfr_tax                  76
gds_profitability_after_tax             71
sales_profit                            61
shop_net_profit_bfr_tax                 61
shop_net_profit_after_tax               61
shop_profitability_after_tax            57
promotion_advertising_wght_avg_rate     47
acquiring_cost_wght_avg_rate            47
cross_docking_cost_wght_avg_rate        42
other_wght_avg_rate                     42
id_x                                    40
position_category                       40
hits_view_search                        40
hits_view                               40
adv_sum_all                             39
hits_view_pdp                           35
date_y                                  34
date_x                                  34
orders_commission                       33
gross_profit                            33
gds_tax                                 32
shop_tax   

In [29]:
# translate to cat
columns = X_train.columns
n_unique = X_train.nunique()

categorical_features = n_unique[n_unique <= 20].index.tolist()
base_features = ['id_x', 'shop_id_x', 'sku', 'id_y', 'shop_id_y', 'date_y']

numeric_features = list(set(columns) - set(categorical_features) - set(base_features))

In [30]:
print("base_features:", len(base_features))
print("categorical_features:", len(categorical_features))
print("numerical_features:", len(numeric_features))

base_features: 6
categorical_features: 38
numerical_features: 25


In [31]:
class ColumnTransformerWarehouse(BaseEstimator, TransformerMixin):
    """
    Creates a onehotencoding features based on `warehouse_name` column
    """
    def __init__(self):
        self.target_column = "warehouse_name"
        self.transform_column = ['quantity_of_paid']
        self.ohe = OneHotEncoder()

    def fit(self, X:pd.DataFrame, y:pd.Series=None):
        return self

    def transform(self, X):
        inf = df_data.groupby(by=['warehouse_name', 'date_y']).agg({'quantity_of_paid': 'sum'}).reset_index()
        new_df = pd.merge(X, inf,  how='left', left_on=['warehouse_name','date_y'], right_on = ['warehouse_name','date_y'])
        return new_df

In [32]:
ct = ColumnTransformerWarehouse()
ct.fit_transform(df_data).head()

# target = quantitty of paid - quantity of return

# offer_id # warehouse # sku # quantity of paid # date # shop_id # position_in cat
# drop sku

,id_x,shop_id_x,date_x,sku,hits_view_search,hits_view_pdp,hits_view,hits_tocart,postings,postings_premium,ordered_units,delivered_units,revenue,cancellations,returns,adv_sum_all,position_category,id_y,shop_id_y,date_y,delivery_schema,warehouse_name,offer_id,title,quantity_of_paid_x,quantity_of_return,accruals_for_orders,accruals_for_return,prime_cost_of_paid,prime_cost_of_return,gross_profit,orders_commission,returns_commission,shipment_processing,direct_logistics,last_mile,cancellation_processing,reverse_logistics,premium_service_cost,installment_sale_cost,seller_bonuses_cost,sales_profit,compensations,compensations_wght_avg_rate,promotion_advertising,promotion_advertising_wght_avg_rate,promotion_reviews,promotion_reviews_wght_avg_rate,cross_docking_cost,cross_docking_cost_wght_avg_rate,placement_cost,placement_cost_wght_avg_rate,acquiring_cost,acquiring_cost_wght_avg_rate,other,other_wght_avg_rate,shop_net_profit_bfr_tax,gds_net_profit_bfr_tax,shop_tax,gds_tax,shop_net_profit_after_tax,gds_net_profit_after_tax,shop_profitability_after_tax,gds_profitability_after_tax,cost_price,fbo_stocks,fbs_stocks,quantity_of_paid_y
0,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202562694,442,63,2491,8,3.0,0.0,3,0,1482.0,0,0,55.71,64.717195,2726400,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-29,FBO,ПУШКИНО,D200,"Бутылка стеклянная, емкость для масла с пробко...",1,0,473.0,0.0,-107.0,0.0,366.0,-66.22,0.0,0.0,-45.0,-26.02,0.0,0.0,0.0,0.0,-1.25,227.51,0.0,0.0,0.0,-11.820688,0.0,0.0,0.0,-9.679734,0.0,0.0,0.0,-4.632474,0.0,-1.567194,227.51,199.809911,-33.11,-33.11,194.40,166.699911,41.10,35.24,107.0,293,0,60
1,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202562694,442,63,2491,8,3.0,0.0,3,0,1482.0,0,0,55.71,64.717195,2726690,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-01,FBO,ПУШКИНО,D200,"Бутылка стеклянная, емкость для масла с пробко...",1,0,473.0,0.0,-107.0,0.0,366.0,-66.22,0.0,0.0,-45.0,-26.02,0.0,0.0,0.0,0.0,0.00,228.76,0.0,0.0,0.0,-16.049131,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,-4.973144,0.0,0.000000,228.76,207.737724,-33.11,-33.11,195.65,174.627724,41.36,36.92,107.0,293,0,180
2,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202562694,442,63,2491,8,3.0,0.0,3,0,1482.0,0,0,55.71,64.717195,2724273,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-01,FBO,НИЖНИЙ НОВГОРОД,D200,"Бутылка стеклянная, емкость для масла с пробко...",5,0,2340.0,0.0,-535.0,0.0,1805.0,-280.80,0.0,0.0,-225.0,-128.70,0.0,0.0,0.0,0.0,-1.25,1169.25,0.0,0.0,0.0,-56.665721,0.0,0.0,0.0,-37.162130,0.0,0.0,0.0,-24.224829,0.0,-8.680882,1169.25,1042.516438,-163.80,-163.80,1005.45,878.716438,42.97,37.55,107.0,293,0,272
3,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202562694,442,63,2491,8,3.0,0.0,3,0,1482.0,0,0,55.71,64.717195,2724274,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-01,FBO,НИЖНИЙ НОВГОРОД,D200,"Бутылка стеклянная, емкость для масла с пробко...",1,0,468.0,0.0,-107.0,0.0,361.0,-56.16,0.0,0.0,-45.0,-25.74,0.0,0.0,0.0,0.0,-1.25,232.85,0.0,0.0,0.0,-11.333144,0.0,0.0,0.0,-7.432426,0.0,0.0,0.0,-4.844966,0.0,-1.736176,232.85,207.503288,-32.76,-32.76,200.09,174.743288,42.75,37.34,107.0,293,0,272
4,10372,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-06-03,202562694,442,63,2491,8,3.0,0.0,3,0,1482.0,0,0,55.71,64.717195,2724275,35996e0d-c3aa-4ad0-b9b6-a64012d5f4fe,2023-05-01,FBO,НИЖНИЙ НОВГОРОД,D200,"Бутылка стеклянная, емкость для масла с пробко...",1,0,468.0,0.0,-107.0,0.0,361.0,-56.16,0.0,0.0,-45.0,-25.74,0.0,0.0,0.0,0.0,0.00,234.10,0.0,0.0,0.0,-11.333144,0.0,0.0,0.0,-7.432426,0.0,0.0,0.0,-4.844966,0.0,-1.736176,234.10,208.753288,-32.76,-32.76,201.34,175.993288,43.02,37.61,107.0,293,0,272


In [34]:
from sklearn.preprocessing import StandardScaler

numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median")), ("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore"))
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)


In [40]:
X_train = X_train.set_index('date_x')
X_test = X_test.set_index('date_x')

In [41]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=10, random_state=0)

from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_test)

# Evaluate the model
score = mean_absolute_error(y_test, preds)
print('MAE:', score)

ValueError: A given column is not a column of the dataframe

# Pipeline

1. Pipeline
2. GridSearchCV
3. ColumnSelector 
4. ColumnTransformer
5. FunctionTransformer
6. FeatureUnion

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
# select single cols
class TextSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]


In [ ]:

text = Pipeline([
                ('selector', TextSelector(key='processed')),
                ('tfidf', TfidfVectorizer( stop_words='english'))
            ])

feats = FeatureUnion([('text', text), 
                      ('length', length),
                      ('words', words),
                      ('words_not_stopword', words_not_stopword),
                      ('avg_word_length', avg_word_length),
                      ('commas', commas)])

feature_processing = Pipeline([('feats', feats)])
feature_processing.fit_transform(X_train)